In [1]:
import pandas as pd
import os
import re
import logging
# import sys
from tkinter import filedialog

# raw_file = open(os.path.normpath("C:/Users/rjaques/PycharmProjects/HMDvalidation/2145-04_Heathrow_SCANNER_2021Lane_1_CL.hmd")
# raw_file = open(os.path.normpath("G:/2019-20 SCANNER Data/WDM/Northern_Ireland/TNI RAV 2019- CL1.HMD"),
# raw_file = open(os.path.normpath("G:/2019-20 SCANNER Data/WDM/Wales/Torfaen/TORFAEN RAV 2019- CR1.HMD"),
# mode='r')

# raw_hmdif = raw_file.readlines()
# raw_file.close()

logger = logging.getLogger('HMDvalidation')
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

curr_dir = os.getcwd()

logger.info('Current directory' + curr_dir)

file_path_variable = filedialog \
    .askdirectory(initialdir=curr_dir, title='Please select a directory containing the data')

if len(file_path_variable) > 0:
    logger.info('Working with ' + file_path_variable)
# return tempdir

# file_list = os.listdir(file_path_variable)
# file_list = glob.glob(file_path_variable)

file_list = list()

file_list += [each for each in os.listdir(file_path_variable) if each.endswith('.hmd')]

print(file_list)


# HMSTART check

# check for only one HMSTART

def line_counting(data_to_process, search_string):
    # Counts the number of lines in a list with the search string in it.

    count = 0

    for line in data_to_process:
        if search_string in line:
            count += 1

    return count


def record_testing(data_to_process, list_of_strings, correct_count):
    # find out if the number of items in a list containing the string is the same as the correct_count

    for field in list_of_strings:
        if line_counting(data_to_process, field) != 1:
            print(field + " is missing")

    return


def check_record_counts(start_string, end_string, file_to_check):
    start_index = [i for i, item in enumerate(file_to_check) if item.startswith(start_string)][0]
    end_index = [i for i, item in enumerate(file_to_check) if item.startswith(end_string)][0]

    length_between_indexes = len(file_to_check[start_index:end_index]) + 1

    res = re.findall(r'\d+', (list(filter(lambda x: end_string in x, raw_hmdif))[0]))

    # When the record count does not equal the value in the *END record shout...

    if int(res[0]) != int(length_between_indexes):
        print('The ' + end_string + ' record count is wrong')

    #    print('Record count OK')

    return


# for file in file_list:
#    print(file)
file = file_path_variable + "/" + file_list[0]

print(file)
# print(os.path.normpath(file[0]))
raw_file = open(file, "r")

raw_hmdif = raw_file.readlines()

raw_file.close()

list_of_records = ['HMSTART', 'TSTART', 'TEND', 'DSTART', 'DEND', 'HMEND']

record_testing(raw_hmdif, list_of_records, 1)

# check the record counts
# using filter() + lambda
# to get string with substring
# res = re.findall(r'\d+', (list(filter(lambda x: 'HMEND' in x, raw_hmdif))[0]))

check_record_counts('HMSTART', 'HMEND', raw_hmdif)
check_record_counts('TSTART', 'TEND', raw_hmdif)
check_record_counts('DSTART', 'DEND', raw_hmdif)

# check for only the values in the list_of_records

raw_hmdif = [w.replace(';', '') for w in raw_hmdif]
raw_hmdif = [w.replace('\n', '') for w in raw_hmdif]
# raw_hmdif = [w.replace('\\', '') for w in raw_hmdif]

# find the DSTART index

start_line = raw_hmdif.index('DSTART')

# all the real data is after DSTART so :

hmd_data = raw_hmdif[(start_line+1):]

2022-06-29 16:45:52,968 - HMDvalidation - INFO - Current directoryC:\Users\royja\DataSpellProjects\HMDvalidation
2022-06-29 16:46:08,865 - HMDvalidation - INFO - Working with C:/Users/royja/DataSpellProjects/Hmd_Validation


['RENFREWSHIRE RAV 2014.hmd']
C:/Users/royja/DataSpellProjects/Hmd_Validation/RENFREWSHIRE RAV 2014.hmd


In [2]:
# print(len(hmd_data))

# remove the control HMEND and DEND records

hmd_data = hmd_data[:-2]

print(len(hmd_data))

list_out = list()
el: str = ""
survey: str = ""
section: str = ""
observation: str = ""

#hmdif_df = pd.DataFrame(hmd_data)

#print(len(hmdif_df))

# list_out = list()
# el: str
# survey: str
for el in hmd_data:
    if el.split('\\')[0] == 'SURVEY':
        survey = el.rsplit('\\', 1)[1]
    if el.split('\\')[0] == 'SECTION':
        section = el.rsplit('\\', 1)[1]
    if el.split('\\')[0] == 'OBSERV':
        observation = el.rsplit('\\', 1)[1]
    if el.split('\\')[0] == 'OBVAL':
        el = el.rsplit('\\', 1)[1]
        # print(f"{survey},{section},{observation},{el}")
        # list_out.append(survey,section,observation,el)
        comma: str = ","
        # seq = str()
        line = ['SURVEY',',', survey,',','SECTION',',', section,',','OBSERV',',',observation,',','OBVAL',',',el]
        seq = " ".join(line)
        # print(seq)
        list_out.append(seq)
        # print(len(list_out))

hmdif_df = pd.DataFrame(list_out)

2141431


In [3]:
# split each record into dataframe columns based on the separating value being a ','
hmdif_df2 = hmdif_df[0].str.split(',', expand=True)

In [4]:

# hmdifDF.columns = hmdifDF.iloc[0]
hmdif_df2.columns = ['SURVEY', 'TYPE', 'VERSION', 'NUMBER', 'SUBSECT', 'MACHINE', 'XSPUSED',
                     'OPERATOR1','OPERATOR2',
                    'SECTION', 'LABEL', 'SNODE', 'LENGTH', 'SDATE',
                   'EDATE', 'STIME', 'ETIME', 'OBSERV', 'DEFECT', 'XSECT', 'SCHAIN',
                   'ECHAIN', 'OBVAL', 'PARM', 'OPTION', 'VALUE', 'PERCENT']
# hmdifDF= hmdifDF[1:]

In [5]:
hmdif_df2['SCHAIN'] = hmdif_df2['SCHAIN'].astype('float')
hmdif_df2['ECHAIN'] = hmdif_df2['ECHAIN'].astype('float')

In [6]:

short_df = hmdif_df2.drop(['SURVEY', 'TYPE', 'VERSION', 'NUMBER', 'SUBSECT',
                         'SECTION', 'SNODE', 'STIME', 'ETIME', 'OBSERV', 'OBVAL'], axis=1)

# short_df.drop_duplicates(inplace=True)

In [7]:
short_df = short_df.round({'SCHAIN':-1, 'ECHAIN':-1})
short_df.drop_duplicates(inplace=True)

In [8]:
short_df['defect_parm'] = short_df['DEFECT'] + short_df['PARM']

short_df.columns

Index(['MACHINE', 'XSPUSED', 'OPERATOR1', 'OPERATOR2', 'LABEL', 'LENGTH',
       'SDATE', 'EDATE', 'DEFECT', 'XSECT', 'SCHAIN', 'ECHAIN', 'PARM',
       'OPTION', 'VALUE', 'PERCENT', 'defect_parm'],
      dtype='object')

In [9]:
short_df2 = short_df.drop_duplicates(keep='last')

#len(short_df)

In [10]:
short_df2.drop(['XSPUSED', 'PARM', 'DEFECT'], axis=1, inplace=True)

dup_index=['MACHINE','OPERATOR1','OPERATOR2', 'LABEL', 'LENGTH', 'SDATE',
             'EDATE', 'XSECT', 'SCHAIN', 'ECHAIN', 'defect_parm']
short_df3 = short_df2.drop_duplicates(subset=dup_index,keep='last')

In [22]:
pivot_index=['MACHINE','OPERATOR1','OPERATOR2', 'LABEL', 'LENGTH', 'SDATE',
             'EDATE', 'XSECT', 'SCHAIN', 'ECHAIN']
pivot_values = ['VALUE']
pivot_column = ['defect_parm']

hmd_pivot = short_df3.pivot(index=pivot_index, columns=pivot_column,values=pivot_values)

In [23]:
#fred = short_df3.pivot(index=pivot_index, columns=pivot_column,values=pivot_values)

#(hmd_pivot.reset_index().index)
# (hmd_pivot.columns.droplevel(0))

mi = hmd_pivot.columns

print(mi)

MultiIndex([('VALUE', ' LCOO 30'),
            ('VALUE', ' LCOO 31'),
            ('VALUE', ' LCOO 32'),
            ('VALUE', ' LCRV 13'),
            ('VALUE', ' LCTM 13'),
            ('VALUE', ' LCTV 13'),
            ('VALUE', ' LECR 14'),
            ('VALUE', ' LEDC 14'),
            ('VALUE', ' LEDR 13'),
            ('VALUE', ' LES1 14'),
            ('VALUE', ' LES2 14'),
            ('VALUE', ' LFAL 14'),
            ('VALUE', ' LGRD 14'),
            ('VALUE', ' LL03 13'),
            ('VALUE', ' LL10 13'),
            ('VALUE', ' LLAD 13'),
            ('VALUE', ' LLBI 13'),
            ('VALUE', ' LLRD 13'),
            ('VALUE', ' LLRT 13'),
            ('VALUE', ' LLTD 13'),
            ('VALUE', ' LLTM 13'),
            ('VALUE', ' LLTV 13'),
            ('VALUE', ' LLTX 13'),
            ('VALUE',  ' LMAP 2'),
            ('VALUE', ' LMAP 23'),
            ('VALUE', ' LMAP 24'),
            ('VALUE', ' LMAP 25'),
            ('VALUE', ' LOVD 14'),
            ('VALUE'

In [24]:
ind = pd.Index([e[1].strip() for e in mi.tolist()])

ind

Index(['LCOO 30', 'LCOO 31', 'LCOO 32', 'LCRV 13', 'LCTM 13', 'LCTV 13',
       'LECR 14', 'LEDC 14', 'LEDR 13', 'LES1 14', 'LES2 14', 'LFAL 14',
       'LGRD 14', 'LL03 13', 'LL10 13', 'LLAD 13', 'LLBI 13', 'LLRD 13',
       'LLRT 13', 'LLTD 13', 'LLTM 13', 'LLTV 13', 'LLTX 13', 'LMAP 2',
       'LMAP 23', 'LMAP 24', 'LMAP 25', 'LOVD 14', 'LR03 13', 'LR10 13',
       'LRAD 13', 'LRBI 13', 'LRCR 14', 'LRRD 13', 'LRRT 13', 'LRTM 13',
       'LRTV 13', 'LSPD 13', 'LSUR 14', 'LT05 13', 'LT95 13', 'LTAD 13',
       'LTRC 14', 'LTRV 13', 'LTVV 13', 'LV10 13', 'LV3 13', 'LV30 13',
       'LWCL 14', 'LWCR 14'],
      dtype='object')

In [25]:
param_list = ind.tolist()
param_columns = pivot_index + param_list
#param_columns.extend(param_list)
#print(param_columns)
#print(pivot_index.append(param_list.to_list()))
#(fred.columns.map(lambda x: '_'.join([str(i) for i in x])))
#(['%s%s' % (a, '|%s' % b if b else '') for a, b in fred.columns])
#([f'{i}|{j}' if j != '' else f'{i}' for i,j in fred.columns])
#fred.index = fred.index(axis=1).values
#fred.index.map(lambda idx: f'{idx[2]}/{idx[1]}')
#fred.reset_index(level = pivot_index)
#fred.to_flat_index()
#(fred.index.get_level_values(1))(fred.index.MultiIndex.nlevels)

#print(fred.head)

In [26]:
(hmd_pivot.reset_index(inplace=True))

In [28]:
hmd_pivot.columns = param_columns

In [21]:
import copy

# i really want a copy...

hmd_ffill2 = copy.copy(hmd_pivot)

# cols = ['LCOO 30', 'LCOO 31', 'LCOO 32']

hmd_ffill2.loc[:,param_list] = hmd_ffill2.loc[:,param_list].ffill(inplace=True)

#hmd_ffill = hmd_ffill.loc[(hmd_ffill['SCHAIN'] == 0) | (hmd_ffill['SCHAIN'] == hmd_ffill['ECHAIN']) ].fillna(method="ffill")
#fred = hmd_ffill.loc[(hmd_ffill.SCHAIN == hmd_ffill.ECHAIN) &
#                    (hmd_ffill.SCHAIN.shift(1) != hmd_ffill.ECHAIN.shift(1)) &
#                     ( (hmd_ffill.SCHAIN.shift(1) == 0) | (hmd_ffill.SCHAIN.shift(-1) == 0)) ].fillna(method="ffill")
#hmd_ffill = hmd_ffill[(hmd_ffill['SCHAIN'] == 0)] #& (hmd_ffill['SCHAIN'] <> hmd_ffill['ECHAIN'])] #= hmd_ffill.fillna(method="bfill")

#hmd_ffill2 = hmd_ffill.xs(['MACHINE',VALUE',axis=1)


In [ ]:
#levels_tuple = ('MACHINE', 'XSPUSED',
 #                'OPERATOR1','OPERATOR2', 'LABEL', 'LENGTH', 'SDATE',
#                 'EDATE', 'XSECT', 'SCHAIN', 'ECHAIN', 'VALUE')
#fred = hmd_ffill.xs('VALUE', axis=1,drop_level=True)

In [19]:
import numpy as np
import copy

hmd_ffill = copy.copy(hmd_pivot)

cols = ['LCOO 30', 'LCOO 31', 'LCOO 32']

#hmd_ffill.loc[:] = \
hmd_ffill.loc[:].ffill(inplace=True, limit=1)


#hmd_ffill.loc[:,cols] = hmd_ffill.loc[:,cols].ffill(inplace=True)

#hmd_ffill2 = hmd_ffill.loc[(hmd_ffill.SCHAIN == 0)]

hmd_ffill['LCOO 30'] = np.where(hmd_ffill.SCHAIN == 0, np.nan, hmd_ffill['LCOO 30'])
hmd_ffill['LCOO 31'] = np.where(hmd_ffill.SCHAIN == 0, np.nan, hmd_ffill['LCOO 31'])
hmd_ffill['LCOO 32'] = np.where(hmd_ffill.SCHAIN == 0, np.nan, hmd_ffill['LCOO 32'])

# remove the rows where SCHAIN = ECHAIN

hmd_ffill = hmd_ffill.loc[hmd_ffill['SCHAIN'] != hmd_ffill['ECHAIN']]
#hmd_ffill = hmd_ffill.loc[(hmd_ffill['SCHAIN'] == 0) | (hmd_ffill['SCHAIN'] == hmd_ffill['ECHAIN']) ].fillna(method="ffill")
#fred = hmd_ffill.loc[(hmd_ffill.SCHAIN == hmd_ffill.ECHAIN) &
#                    (hmd_ffill.SCHAIN.shift(1) != hmd_ffill.ECHAIN.shift(1)) &
#                     ( (hmd_ffill.SCHAIN.shift(1) == 0) | (hmd_ffill.SCHAIN.shift(-1) == 0)) ].fillna(method="ffill")
#hmd_ffill = hmd_ffill[(hmd_ffill['SCHAIN'] == 0)] #& (hmd_ffill['SCHAIN'] <> hmd_ffill['ECHAIN'])] #= hmd_ffill.fillna(method="bfill")

#hmd_ffill2 = hmd_ffill.xs(['MACHINE',VALUE',axis=1)

In [ ]:
# git test

In [ ]:
# new cell to check save

In [ ]:
# git test

In [24]:
# new cell to check save